<a href="https://colab.research.google.com/github/rick1270/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/RClayton_LS_DS1_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [1]:
!pip install kaggle

In [4]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 40.4MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.7MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.02MB/s]


In [5]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [6]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [7]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

0.08389810726550917

In [8]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([0.79543117, 0.58610823, 0.67595121, 0.793829  , 0.62090522,
       0.72542107, 0.59848968, 0.58734245, 0.48567063, 0.77627736,
       0.84211887, 0.57052439, 0.7754689 , 0.96621114, 0.70287941,
       0.57673837, 0.72321391, 0.75894755, 0.77968041, 0.50246003,
       0.49858077, 0.7474959 , 0.3542282 , 0.61648435, 0.71300224,
       0.66294608, 0.53175333, 0.77397395, 0.68419387, 0.68395536,
       0.52041202, 0.56814038, 0.79586606, 0.81372012, 0.61068545,
       0.57260627, 0.52525981, 0.58055388, 0.45584728, 0.67976208,
       0.8226707 , 0.84286197, 0.96189157, 0.66724612, 0.68589478,
       0.61846513, 0.63455044, 0.68275686, 0.65738372, 0.45198998,
       0.59988596, 0.63845908, 0.63132487, 0.7888473 , 0.60126246,
       0.79714045, 0.78713803, 0.54643775, 0.42823635, 0.7711724 ,
       0.53552976, 0.55608044, 0.54480459, 0.57031915, 0.65080369,
       0.77958926, 0.6371013 , 0.70993488, 0.71493598, 0.60375943,
       0.54407206, 0.48186138, 0.76576089, 0.75456305, 0.53968

In [9]:
linear_reg.coef_

array([-0.08596295, -0.00829314,  0.00048775])

In [10]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

In [11]:
np.dot(linear_reg.coef_, test_case.reshape(-1, 1)) + linear_reg.intercept_

array([1.14845883])

In [12]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7103825136612022

In [13]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
log_reg.predict(test_case)[0]

1

In [15]:
help(log_reg.predict)

Help on method predict in module sklearn.linear_model.base:

predict(X) method of sklearn.linear_model.logistic.LogisticRegression instance
    Predict class labels for samples in X.
    
    Parameters
    ----------
    X : array_like or sparse matrix, shape (n_samples, n_features)
        Samples.
    
    Returns
    -------
    C : array, shape [n_samples]
        Predicted class label per sample.



In [16]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [17]:
# What's the math?
log_reg.coef_

array([[-0.0455017 , -0.02912513,  0.0048037 ]])

In [18]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [20]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [21]:
# Live - let's try absenteeism!
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
!unzip Absenteeism_at_work_AAA.zip	

--2019-01-23 21:32:13--  http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66136 (65K) [application/zip]
Saving to: ‘Absenteeism_at_work_AAA.zip’

Absenteeism_at_work 100%[===================>]  64.59K   179KB/s    in 0.4s    

2019-01-23 21:32:14 (179 KB/s) - ‘Absenteeism_at_work_AAA.zip’ saved [66136/66136]

Archive:  Absenteeism_at_work_AAA.zip
  inflating: Absenteeism_at_work.arff  
  inflating: Absenteeism_at_work.csv  
  inflating: Absenteeism_at_work.xls  
  inflating: Attribute Information.docx  
  inflating: UCI_ABS_TEXT.docx       


In [22]:
!head Absenteeism_at_work.csv

ID;Reason for absence;Month of absence;Day of the week;Seasons;Transportation expense;Distance from Residence to Work;Service time;Age;Work load Average/day ;Hit target;Disciplinary failure;Education;Son;Social drinker;Social smoker;Pet;Weight;Height;Body mass index;Absenteeism time in hours
11;26;7;3;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;4
36;0;7;3;1;118;13;18;50;239.554;97;1;1;1;1;0;0;98;178;31;0
3;23;7;4;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
7;7;7;5;1;279;5;14;39;239.554;97;0;1;2;1;1;0;68;168;24;4
11;23;7;5;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;2
3;23;7;6;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
10;22;7;6;1;361;52;3;28;239.554;97;0;1;1;1;0;4;80;172;27;8
20;23;7;6;1;260;50;11;36;239.554;97;0;1;4;1;0;0;65;168;23;4
14;19;7;2;1;155;12;14;34;239.554;97;0;1;2;1;0;0;95;196;25;40


In [23]:
absent_df = pd.read_table('Absenteeism_at_work.csv', sep=';')
absent_df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [24]:
absent_df.shape

(740, 21)

In [25]:
LogisticRegression

sklearn.linear_model.logistic.LogisticRegression

In [26]:
absent_df.columns

Index(['ID', 'Reason for absence', 'Month of absence', 'Day of the week',
       'Seasons', 'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target',
       'Disciplinary failure', 'Education', 'Son', 'Social drinker',
       'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours'],
      dtype='object')

In [27]:
X = absent_df.drop('Reason for absence', axis='columns')
y = absent_df['Reason for absence']

absent_log1 = LogisticRegression().fit(X, y)
absent_log1.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5013513513513513

In [28]:
absent_log1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [29]:
absent_log1.coef_

array([[-1.27508146e-02, -2.20793886e-01, -4.08404457e-02,
         4.22616389e-01,  7.64613632e-03, -6.18658809e-03,
        -3.11523480e-02,  5.52309565e-02, -1.01707258e-02,
        -1.14781041e-01,  2.51984519e+00,  1.41303285e-02,
         3.56702636e-01,  2.27778616e-01,  1.80478520e-01,
         1.33441356e-01, -9.82098540e-03,  5.23702470e-02,
         7.24744784e-02, -3.15221513e+00],
       [ 5.11128074e-02,  1.77166203e-01, -5.01201750e-01,
        -9.34413067e-02,  8.70070989e-03,  2.90683453e-02,
        -8.19496921e-02,  1.30910624e-01,  3.69193210e-03,
         3.58478924e-02, -7.77198534e-01,  9.21419989e-01,
         2.74046853e-01, -6.53744541e-01, -2.74959514e-01,
        -4.46143318e-01,  1.88638790e-01, -7.60409657e-02,
        -6.85430554e-01,  4.55734002e-03],
       [ 1.74477093e-01,  5.91768593e-01,  4.46355493e-02,
         4.29016838e-01,  5.97802716e-02, -2.02281583e-01,
        -4.44906133e-01, -2.65239556e-01,  8.55486140e-03,
        -4.07755000e-01, -6.7

In [30]:
absent_log1.predict(X)

array([23,  0, 23, 25, 23, 23, 22, 28, 18, 25, 28, 28, 28, 23, 23, 28, 23,
       11, 23, 23, 23, 23, 22, 28, 24, 22, 23, 28, 23, 23, 28, 18, 23, 18,
       23, 23, 23, 28, 18, 28, 23, 23, 23, 26, 23, 23, 23, 23, 28, 13,  0,
        0, 23, 23,  0,  0, 23, 28,  0, 23, 28, 28, 28, 23,  0, 23, 28, 23,
       23, 23, 23, 28, 28, 23, 19, 28, 23, 24, 23, 28, 28, 23, 13, 21, 28,
       10, 23, 13, 23, 23, 23, 21, 23, 28, 23, 23, 13, 28, 23, 13, 23, 23,
       23, 28, 13, 22, 23, 28, 23, 22, 23, 28, 23, 23, 27, 27, 18, 23, 27,
       23, 27, 27, 23, 27, 27, 27, 27, 27, 27, 13, 22, 27, 27, 27, 27, 23,
       22, 27, 28, 13, 23, 22, 28, 22,  1,  1, 23, 23, 27, 11, 23, 11, 23,
       23, 22, 28, 28, 28, 17, 13, 28, 13, 23, 13, 11, 18, 13, 18, 22, 17,
       28, 23,  7, 13, 23, 13, 13, 13, 13, 28, 14, 23, 23, 23, 28, 23, 28,
       13, 23, 23, 28, 13, 28, 28, 28, 28, 14, 13, 13, 10, 22, 13, 22,  0,
       13, 28, 28, 13, 13, 28, 28, 28, 13,  0,  0,  0,  0, 28, 13, 22, 13,
       22, 13, 28, 13, 13

In [31]:
absent_log1.predict_proba(X)[0]

array([2.69982649e-06, 6.91615398e-03, 8.57701135e-09, 5.16203311e-08,
       2.18856804e-07, 5.92457902e-08, 2.75125708e-03, 4.07323392e-03,
       1.96525568e-03, 1.28991052e-10, 1.04724148e-02, 2.54302198e-02,
       9.93451673e-05, 3.60953076e-02, 1.02877921e-02, 1.48667629e-04,
       1.20483308e-07, 4.78991713e-21, 2.50188861e-02, 8.82617162e-02,
       2.20255602e-04, 1.37766587e-01, 3.10018914e-01, 1.68568210e-02,
       5.54127149e-02, 1.77084959e-01, 1.38810366e-06, 9.11149514e-02])

In [0]:
?LogisticRegression

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [33]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-01-23 21:40:08--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  25.8MB/s    in 14s     

2019-01-23 21:40:23 (24.4 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

In [57]:
# Thanks A!
tracks = pd.read_csv('fma_metadata/tracks.csv', header=[0,1])
tracks.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Unnamed: 0_level_0    album                                            \
  Unnamed: 0_level_1 comments         date_created        date_released   
0           track_id      NaN                  NaN                  NaN   
1                  2      0.0  2008-11-26 01:44:45  2009-01-05 00:00:00   
2                  3      0.0  2008-11-26 01:44:45  2009-01-05 00:00:00   
3                  5      0.0  2008-11-26 01:44:45  2009-01-05 00:00:00   
4                 10      0.0  2008-11-26 01:45:08  2008-02-06 00:00:00   

                                                              \
  engineer favorites   id information  listens producer tags   
0      NaN       NaN  NaN         NaN      NaN      NaN  NaN   
1      NaN       4.0  1.0     <p></p>   6073.0      NaN   []   
2      NaN       4.0  1.0     <p></p>   6073.0      NaN   []   
3      NaN       4.0  1.0     <p></p>   6073.0      NaN   []   
4      NaN       4.0  6.0         NaN  47632.0      NaN   []   

                                                    artist                  \
                  title tracks   type    active_year_begin active_year_end   
0                   NaN    NaN    NaN                  NaN             NaN   
1  AWOL - A Way Of Life    7.0  Album  2006-01-01 00:00:00             NaN   
2  AWOL - A Way Of Life    7.0  Album  2006-01-01 00:00:00             NaN   
3  AWOL - A Way Of Life    7.0  Album  2006-01-01 00:00:00             NaN   
4     Constant Hitmaker    2.0  Album                  NaN             NaN   

                                                      \
                                   associated_labels   
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  Mexican Summer, Richie Records, Woodsist, Skul...   

                                                               \
                                                 bio comments   
0                                                NaN      NaN   
1  <p>A Way Of Life, A Collective of Hip-Hop from...      0.0   
2  <p>A Way Of Life, A Collective of Hip-Hop from...      0.0   
3  <p>A Way Of Life, A Collective of Hip-Hop from...      0.0   
4  <p><span style="font-family:Verdana, Geneva, A...      3.0   

                                                                         \
          date_created favorites   id   latitude    location  longitude   
0                  NaN       NaN  NaN        NaN         NaN        NaN   
1  2008-11-26 01:42:32       9.0  1.0  40.058324  New Jersey -74.405661   
2  2008-11-26 01:42:32       9.0  1.0  40.058324  New Jersey -74.405661   
3  2008-11-26 01:42:32       9.0  1.0  40.058324  New Jersey -74.405661   
4  2008-11-26 01:42:55      74.0  6.0        NaN         NaN        NaN   

                                                                 \
                                             members       name   
0                                                NaN        NaN   
1  Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...       AWOL   
2  Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...       AWOL   
3  Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...       AWOL   
4                           Kurt Vile, the Violators  Kurt Vile   

                                                                               \
                                    related_projects                     tags   
0                                                NaN                      NaN   
1  The list of past projects is 2 long but every1...                 ['awol']   
2  The list of past projects is 2 long but every1...                 ['awol']   
3  The list of past projects is 2 long but every1...                 ['awol']   
4                                                NaN  ['philly', 'kurt vile']   

                                                                set

In [0]:
# Maybe it'd be handy to sample so we have less data so things run faster
tracks = tracks.sample(2000, random_state=42)

In [37]:
tracks.describe()

album                                                       \
          comments    favorites           id       listens       tracks   
count  2000.000000  2000.000000   2000.00000  2.000000e+03  2000.000000   
mean      0.443500     1.285500  12850.28600  3.739742e+04    19.552500   
std       2.884262     3.086868   6219.58821  1.964804e+05    35.496877   
min      -1.000000    -1.000000     -1.00000 -1.000000e+00    -1.000000   
25%       0.000000     0.000000   7895.00000  3.309250e+03     7.000000   
50%       0.000000     0.000000  13370.00000  8.911500e+03    11.000000   
75%       0.000000     1.000000  18232.75000  2.463975e+04    18.000000   
max      53.000000    55.000000  22905.00000  3.564243e+06   652.000000   

            artist                                                     \
          comments    favorites            id    latitude   longitude   
count  2000.000000  2000.000000   2000.000000  836.000000  836.000000   
mean      2.218000    33.792500  12022.082000   38.662066  -34.336361   
std       7.181244   113.006777   6791.986517   20.769404   69.277985   
min      -1.000000    -1.000000      4.000000  -45.878760 -123.114034   
25%       0.000000     1.000000   6567.500000   39.243541  -80.289438   
50%       0.000000     5.000000  12038.500000   41.846417  -72.713299   
75%       1.000000    16.250000  17690.000000   48.938791    5.369780   
max      79.000000   951.000000  24296.000000   64.963051  174.885971   

               track                                                       \
            bit_rate     comments     duration    favorites      interest   
count    2000.000000  2000.000000  2000.000000  2000.000000  2.000000e+03   
mean   265349.416500     0.037000   282.718000     3.144000  4.299708e+03   
std     66587.921464     0.371079   295.134104    16.531119  3.374043e+04   
min        -1.000000     0.000000     1.000000     0.000000  7.000000e+00   
25%    196496.000000     0.000000   152.000000     0.000000  5.847500e+02   
50%    320000.000000     0.000000   220.000000     1.000000  1.247500e+03   
75%    320000.000000     0.000000   308.000000     3.000000  3.099250e+03   
max    320817.000000    11.000000  3710.000000   599.000000  1.314156e+06   

                                   
             listens       number  
count    2000.000000  2000.000000  
mean     2567.795000     8.137000  
std     12456.774413    13.616899  
min         3.000000     0.000000  
25%       264.000000     2.000000  
50%       729.000000     5.000000  
75%      2031.250000     9.000000  
max    429168.000000   206.000000

In [55]:
pd.set_option('display.max_columns', None)  # Unlimited columns
tracks.sample(5)

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
20834           0  2009-11-12 05:23:00                  NaN      NaN   
61153           0  2012-02-27 02:18:54  2004-01-01 00:00:00      NaN   
12869           0  2009-05-14 12:00:56  2007-01-12 00:00:00      NaN   
61162           1  2012-02-27 14:29:42                  NaN      NaN   
41741           0  2011-01-03 16:00:09  2010-07-03 00:00:00      NaN   

                                                                              \
         favorites     id                                        information   
track_id                                                                       
20834            0   4661                                                NaN   
61153            5  11170  <p>\n</p><p>Anarchist Christian group has shar...   
12869            0   3083  <div class="album_page_description">\n<p>A Kur...   
61162            0  11173                                                NaN   
41741            0   8220  <p><span class="Apple-style-span" style="borde...   

                                                                          \
         listens producer tags                              title tracks   
track_id                                                                   
20834       7840      NaN   []  Dark Cabinet of the Strange Weird     11   
61153      54904      NaN   []                           us vs us     14   
12869       3879      NaN   []                       Deer's Curse     13   
61162      17348      NaN   []                    This Is America     19   
41741       1721      NaN   []                   Telar Deslizante     12   

                           artist                                    \
           type active_year_begin active_year_end associated_labels   
track_id                                                              
20834     Album               NaN             NaN               NaN   
61153     Album               NaN             NaN               NaN   
12869     Album               NaN             NaN               NaN   
61162     Album               NaN             NaN               NaN   
41741     Album               NaN             NaN               NaN   

                                                                      \
                                                        bio comments   
track_id                                                               
20834                                                   NaN        0   
61153     <p>Anarchist Christian group has shared the st...        0   
12869     <p>He has composed music for over 80 theater p...        0   
61162                                                   NaN        1   
41741                                                   NaN        1   

                                                           \
                 date_created favorites     id   latitude   
track_id                                                    
20834     2009-11-12 05:23:05         2   5496        NaN   
61153     2012-02-27 02:14:09        19  13065  39.952280   
12869     2009-05-14 11:58:30         2   3459        NaN   
61162     2011-10-24 12:15:26        38  12085  45.512280   
41741     2011-01-03 16:02:21         2   9824 -37.020664   

                                                  \
                             location  longitude   
track_id                                           
20834                             NaN        NaN   
61153                    Philadelphia -75.162454   
12869                             NaN        NaN   
61162     Montreal, Quebec Bas-Canada -73.554390   
41741                           Chile -87.946239   

                                                             \
                                                    members   
track_i

In [52]:
tracks.dtypes

album   comments               int64
        date_created          object
        date_released         object
        engineer              object
        favorites              int64
        id                     int64
        information           object
        listens                int64
        producer              object
        tags                  object
        title                 object
        tracks                 int64
        type                  object
artist  active_year_begin     object
        active_year_end       object
        associated_labels     object
        bio                   object
        comments               int64
        date_created          object
        favorites              int64
        id                     int64
        latitude             float64
        location              object
        longitude            float64
        members               object
        name                  object
        related_projects      object
 

In [39]:
pd.options.mode.use_inf_as_na = True
tracks.isna().sum()

album   comments                0
        date_created           65
        date_released         680
        engineer             1729
        favorites               0
        id                      0
        information           436
        listens                 0
        producer             1682
        tags                    0
        title                  22
        tracks                  0
        type                  112
artist  active_year_begin    1588
        active_year_end      1903
        associated_labels    1722
        bio                   678
        comments                0
        date_created           14
        favorites               0
        id                      0
        latitude             1164
        location              703
        longitude            1164
        members              1134
        name                    0
        related_projects     1751
        tags                    0
        website               544
        wikipe

In [42]:


# Another approach, restrict columns by type
# Shreyas
numeric = ['int32', 'int64', 'float16', 'float32', 'float64']
tracks_num = tracks.select_dtypes(include=numeric).copy()
tracks.select_dtypes('number')
tracks._get_numeric_data()  # "inner" method shorthand

# Another thing you can do - sample! See above where data loaded

album                                   artist                   \
         comments favorites     id listens tracks comments favorites     id   
track_id                                                                      
43742          -1        -1   8578      -1     -1        4        82   8390   
31941           0         1   6655   19141     14        1        15   7888   
50985           3        10   9619  458874     20       32       431  11431   
146981          0         0  21917   19158     11        4       110  15891   
20461           0         1   4611    3139      5        1         6   5443   
66124           0         1  11883    4642     11        0         8   1905   
54231           1         3  10138   39825      9        0         8  11955   
37272           4         6   7507   76096    154        0         0   8925   
91061           0         1  15083    4492      8        0         3  17017   
89882           0         2  14966   10262     16        0         0  16908   
25727           0         1   5581    2660     22        0         5   6572   
79535           0         1  13655    4514      9       12        86  10341   
61870           1         1  11237   13949     18        1         8  13173   
98602          -1        -1  15950      -1     -1        1         9  15196   
63234           0         0  11461   22620     11        0         5  13343   
35827           0         0   7341    5380     32        0         1   8632   
21207           1         0   4721    3706      8        0         0   5562   
129525          0         0  19497     796     44        0         0  21284   
81654           0        11  13930  271279     25       29       784   7168   
1237            1         1    326    7559     13        0        21    219   
139988          0         3  21067   78615     19        2        26  22407   
106698          0         1  16623    1809      5        0         6   3226   
145495          0         1  21739   47905     21        0         1  19527   
115820         -1        -1  17807      -1     -1        1         8  17402   
85758           0         0  14455    7089      5        0         4  11577   
99511           0         1  16075   10650     13        1         4  12932   
126854          0         0  19177    9322     12        0         1  21038   
66875           0         4  11990   23405     42        3         9  13912   
43486           4         7   8533  121412     18        0         0  10160   
154883          0         0  22859    4919     15        4       110  15891   
...           ...       ...    ...     ...    ...      ...       ...    ...   
155115          1         0  22905   11141     10       55       861  16852   
79796           0         0  13698    6824      2        7        11   4634   
84591           0         3  14291   20640     27        0         0  16322   
1093            0         0    299    2453     10        1         2    202   
113215          0         0  17448   32079     14        0         5  19030   
37739           0         1   7536    1877     16        0         4   8990   
147590          0         2  22003   27143     88        0         0  23440   
89781           1         0  14959   11188     37        1         7   1640   
26957          -1        -1   5830      -1     -1        0        23    966   
29588           0         0   6312    3938      9        0         0   7368   
90902           0         0  15052    5588     12        0         1  17011   
49813           0         0   9460   18770      7        0         0  11298   
136274          0         0  20468    6201      3        0         0   4368   
87968           0         0  14793    6530     10        0         0  16824   
31554           0         1   6607     205      3        1         3   2194   
70166           0         0  12468    5258     12        0         4   5144   
108255          0         0  16838   12129     16        1    

In [43]:
tracks._get_numeric_data()

album                                   artist                   \
         comments favorites     id listens tracks comments favorites     id   
track_id                                                                      
43742          -1        -1   8578      -1     -1        4        82   8390   
31941           0         1   6655   19141     14        1        15   7888   
50985           3        10   9619  458874     20       32       431  11431   
146981          0         0  21917   19158     11        4       110  15891   
20461           0         1   4611    3139      5        1         6   5443   
66124           0         1  11883    4642     11        0         8   1905   
54231           1         3  10138   39825      9        0         8  11955   
37272           4         6   7507   76096    154        0         0   8925   
91061           0         1  15083    4492      8        0         3  17017   
89882           0         2  14966   10262     16        0         0  16908   
25727           0         1   5581    2660     22        0         5   6572   
79535           0         1  13655    4514      9       12        86  10341   
61870           1         1  11237   13949     18        1         8  13173   
98602          -1        -1  15950      -1     -1        1         9  15196   
63234           0         0  11461   22620     11        0         5  13343   
35827           0         0   7341    5380     32        0         1   8632   
21207           1         0   4721    3706      8        0         0   5562   
129525          0         0  19497     796     44        0         0  21284   
81654           0        11  13930  271279     25       29       784   7168   
1237            1         1    326    7559     13        0        21    219   
139988          0         3  21067   78615     19        2        26  22407   
106698          0         1  16623    1809      5        0         6   3226   
145495          0         1  21739   47905     21        0         1  19527   
115820         -1        -1  17807      -1     -1        1         8  17402   
85758           0         0  14455    7089      5        0         4  11577   
99511           0         1  16075   10650     13        1         4  12932   
126854          0         0  19177    9322     12        0         1  21038   
66875           0         4  11990   23405     42        3         9  13912   
43486           4         7   8533  121412     18        0         0  10160   
154883          0         0  22859    4919     15        4       110  15891   
...           ...       ...    ...     ...    ...      ...       ...    ...   
155115          1         0  22905   11141     10       55       861  16852   
79796           0         0  13698    6824      2        7        11   4634   
84591           0         3  14291   20640     27        0         0  16322   
1093            0         0    299    2453     10        1         2    202   
113215          0         0  17448   32079     14        0         5  19030   
37739           0         1   7536    1877     16        0         4   8990   
147590          0         2  22003   27143     88        0         0  23440   
89781           1         0  14959   11188     37        1         7   1640   
26957          -1        -1   5830      -1     -1        0        23    966   
29588           0         0   6312    3938      9        0         0   7368   
90902           0         0  15052    5588     12        0         1  17011   
49813           0         0   9460   18770      7        0         0  11298   
136274          0         0  20468    6201      3        0         0   4368   
87968           0         0  14793    6530     10        0         0  16824   
31554           0         1   6607     205      3        1         3   2194   
70166           0         0  12468    5258     12        0         4   5144   
108255          0         0  16838   12129     16        1    

In [44]:
tracks.shape

(2000, 52)

In [0]:
X = absent_df.drop('Reason for absence', axis='columns')
y = absent_df['Reason for absence']

absent_log1 = LogisticRegression().fit(X, y)
absent_log1.score(X, y)

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.